<a href="https://colab.research.google.com/github/rdv0777/AB-Test/blob/main/%D0%A2%D0%97_%D0%A0%D0%B0%D0%B7%D0%B2%D0%BE%D0%B4%D0%BE%D0%B2_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
from scipy import stats
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df = pd.read_csv('/content/test_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Campaign_id        87 non-null     int64 
 1   Geo                87 non-null     object
 2   Buyer              87 non-null     object
 3   Impressions        87 non-null     int64 
 4   Clicks             87 non-null     int64 
 5   Installs           87 non-null     int64 
 6   Registrations      87 non-null     int64 
 7   Spend              87 non-null     object
 8   Revenue_ad1d       87 non-null     object
 9   Revenue_in_app1d   87 non-null     object
 10  Payers1d           87 non-null     int64 
 11  Revenue_ad7d       87 non-null     object
 12  Revenue_in_app7d   87 non-null     object
 13  Payers7d           87 non-null     int64 
 14  Revenue_ad30d      87 non-null     object
 15  Revenue_in_app30d  87 non-null     object
 16  Payers30d          87 non-null     int64 
dtyp

In [66]:
df['Spend'] = df['Spend'].apply(lambda x: int(x.split()[0].replace(',','')))


In [67]:
df['Revenue_ad1d'] = df['Revenue_ad1d'].apply(lambda x: int(x.split()[0].replace(',','')))
df['Revenue_ad7d'] = df['Revenue_ad7d'].apply(lambda x: int(x.split()[0].replace(',','')))
df['Revenue_ad30d'] = df['Revenue_ad30d'].apply(lambda x: int(x.split()[0].replace(',','')))
df['Revenue_in_app1d'] = df['Revenue_in_app1d'].apply(lambda x: int(x.split()[0].replace(',','')))
df['Revenue_in_app7d'] = df['Revenue_in_app7d'].apply(lambda x: int(x.split()[0].replace(',','')))
df['Revenue_in_app30d'] = df['Revenue_in_app30d'].apply(lambda x: int(x.split()[0].replace(',','')))

In [68]:
df.describe()



,Campaign_id,Impressions,Clicks,Installs,Registrations,Spend,Revenue_ad1d,Revenue_in_app1d,Payers1d,Revenue_ad7d,Revenue_in_app7d,Payers7d,Revenue_ad30d,Revenue_in_app30d,Payers30d
count,87.000000,8.700000e+01,87.000000,87.000000,87.000000,8.700000e+01,8.700000e+01,8.700000e+01,87.000000,8.700000e+01,8.700000e+01,87.000000,8.700000e+01,8.700000e+01,87.000000
mean,44.000000,1.140572e+05,3804.494253,1211.850575,769.057471,1.706217e+08,7.402273e+07,2.372510e+06,2.034483,1.127267e+08,4.943219e+06,2.551724,1.275146e+08,6.803389e+06,2.816092
std,25.258662,3.989255e+05,14610.898115,3883.087208,2577.564690,4.297927e+08,1.643234e+08,1.765933e+07,7.169782,2.314334e+08,2.340131e+07,8.798465,2.756012e+08,3.308662e+07,9.299323
min,1.000000,2.185000e+03,53.000000,10.000000,2.000000,2.100000e+01,2.121650e+05,0.000000e+00,0.000000,1.927960e+05,0.000000e+00,0.000000,3.686630e+05,0.000000e+00,0.000000
25%,22.500000,1.057700e+04,276.500000,107.500000,44.500000,6.058500e+03,6.639435e+06,0.000000e+00,0.000000,1.060429e+07,0.000000e+00,0.000000,1.140460e+07,0.000000e+00,0.000000
50%,44.000000,2.619300e+04,724.000000,258.000000,147.000000,2.463000e+07,1.689081e+07,0.000000e+00,0.000000,2.202928e+07,0.000000e+00,0.000000,2.685011e+07,4.900000e+01,1.000000
75%,65.500000,7.088950e+04,2063.500000,763.500000,501.500000,1.411400e+08,7.767586e+07,3.830000e+02,2.000000,1.113115e+08,4.020000e+02,2.000000,1.127409e+08,8.415000e+02,3.000000
max,87.000000,3.456974e+06,129508.000000,33054.000000,21932.000000,3.208440e+09,1.163521e+09,1.622200e+08,64.000000,1.381310e+09,1.997500e+08,79.000000,1.527014e+09,2.766300e+08,83.000000


In [69]:

df['Adver_revenue'] = df['Revenue_ad1d'] + df['Revenue_ad7d'] + df['Revenue_ad30d']
df.Adver_revenue = df.Adver_revenue.astype('int')
df['Rev_purchases'] = df['Revenue_in_app1d'] + df['Revenue_in_app7d'] + df['Revenue_in_app30d']
df.Rev_purchases = df.Rev_purchases.astype('int')
df['Tot_rev'] = df['Adver_revenue'] + df['Rev_purchases']
df.Tot_rev = df.Tot_rev.astype('int')
df.head()

,Campaign_id,Geo,Buyer,Impressions,Clicks,Installs,Registrations,Spend,Revenue_ad1d,Revenue_in_app1d,Payers1d,Revenue_ad7d,Revenue_in_app7d,Payers7d,Revenue_ad30d,Revenue_in_app30d,Payers30d,Adver_revenue,Rev_purchases,Tot_rev
0,1,north america,buyer_1,3456974,129508,33054,21932,3208440002,1163521408,162220001,64,1381310062,199749999,79,1527013835,276629998,83,4071845305,638599998,4710445303
1,2,north america,buyer_2,245542,6306,3401,2448,1569519999,833059639,3725,17,1200964697,69340001,19,1449001828,120250001,22,3483026164,189593727,3672619891
2,3,north america,buyer_1,1281103,35724,13315,8101,1231999998,4596826,326,4,524987826,247,6,572633765,247,6,1102218417,820,1102219237
3,4,north america,buyer_1,215538,9987,3119,1857,1228129999,60277816,2613,11,830113889,33439999,12,1042831595,57530002,12,1933223300,90972614,2024195914
4,5,western europe,buyer_2,335878,7343,3952,2955,872570005,427807238,19030001,4,577039081,20640001,5,714554179,32150001,6,1719400498,71820003,1791220501


In [70]:
df['Payback_media_bayer'] = df['Spend'] / df['Tot_rev']
df.head()

,Campaign_id,Geo,Buyer,Impressions,Clicks,Installs,Registrations,Spend,Revenue_ad1d,Revenue_in_app1d,...,Revenue_ad7d,Revenue_in_app7d,Payers7d,Revenue_ad30d,Revenue_in_app30d,Payers30d,Adver_revenue,Rev_purchases,Tot_rev,Payback_media_bayer
0,1,north america,buyer_1,3456974,129508,33054,21932,3208440002,1163521408,162220001,...,1381310062,199749999,79,1527013835,276629998,83,4071845305,638599998,4710445303,0.681133
1,2,north america,buyer_2,245542,6306,3401,2448,1569519999,833059639,3725,...,1200964697,69340001,19,1449001828,120250001,22,3483026164,189593727,3672619891,0.427357
2,3,north america,buyer_1,1281103,35724,13315,8101,1231999998,4596826,326,...,524987826,247,6,572633765,247,6,1102218417,820,1102219237,1.117745
3,4,north america,buyer_1,215538,9987,3119,1857,1228129999,60277816,2613,...,830113889,33439999,12,1042831595,57530002,12,1933223300,90972614,2024195914,0.606725
4,5,western europe,buyer_2,335878,7343,3952,2955,872570005,427807238,19030001,...,577039081,20640001,5,714554179,32150001,6,1719400498,71820003,1791220501,0.487137


In [72]:
df.sort_values(['Payback_media_bayer'],ascending=False).groupby(['Buyer']).agg({'Payback_media_bayer': ['mean']})


,Payback_media_bayer
,mean
Buyer,
buyer_1,0.761186
buyer_2,0.882271
buyer_3,0.152510
buyer_4,0.559900
buyer_5,1.998804
